# Update Items

### Import Libraries

In [ ]:
import glob
import os
import json
import xml.etree.ElementTree as ET
import requests
from urllib import parse
import argparse
import urllib.request
import configparser
from tqdm import tqdm
import time

### Get configuration file

In [548]:
config = configparser.ConfigParser()
config.sections()
config.read('../config/api.ini')
client_id = config.get('main', 'client_id')
client_secret = config.get('main', 'client_secret')
endpoint = config.get('main', 'endpoint')

### Connect to the Api

In [549]:
params = {
    'key_identity': client_id,
    'key_credential': client_secret
}

access_url = endpoint
print("Connected to:    ", access_url)

Connected to:     http://localhost:8080/api/


### Upload Images

In [622]:
import glob
url = endpoint + 'items?page=all'

print(url)
response = json.loads(urllib.request.urlopen(url).read().decode('utf-8'))

pbar = tqdm(range(len(response)), desc='description')

for i in pbar:
    try:
        if response[i]['crm:P1_is_identified_by'][0]['@value']:
            identifier = response[i]['crm:P1_is_identified_by'][0]['@value']
        else:
            identifier = response[i]['o:title'][0]['@value']

        resourceId = response[i]['o:id']
        images = [f for f in glob.glob("../media/mid/*.jpg") if identifier in f]
        
        for pos, image in enumerate(images):
            
            data = {
                "o:ingester": "upload", 
                "file_index": pos, 
                "o:item": {"o:id": resourceId }
            }

            files = [
                 ("data", (None, json.dumps(data), "application/json")),
                 ("file["+str(pos)+"]", (image, open(image, "rb"), "image/jpg"))
            ]   
            
            post = requests.post(endpoint + 'media', params=params, files=files)

            time.sleep(.25)
            
        break
            
    except Exception as e:
        print(e)
            
            

http://localhost:8080/api/items?page=all


description:   0%|          | 0/641 [00:05<?, ?it/s]


In [584]:
item = 687 # The number according to the omeka s resource

In [585]:
image = 'A_D0238D3_M_186_555_KarteiKopieWeiss.jpg'
imageType = 'jpg'

In [586]:
data = {
    "o:ingester": "upload", 
    "file_index": "0", 
    "o:item": {"o:id": item }
}

files = [
     ('data', (None, json.dumps(data), 'application/json')),
     ('file[0]', (image, open(image, 'rb'), 'image/' + imageType))
]

response = requests.post(endpoint + '/media', params=params, files=files)

